# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [82]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [83]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,38.336,87,97,10.29,TF,08/10/2023
1,1,mersa matruh,31.3525,27.2453,77.522,71,0,2.57,EG,08/10/2023
2,2,northumberland,55.2500,-2.0000,58.892,95,13,2.82,GB,08/10/2023
3,3,bredasdorp,-34.5322,20.0403,49.658,87,0,1.81,ZA,08/10/2023
4,4,puerto natales,-51.7236,-72.4875,35.978,94,84,1.61,CL,08/10/2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [84]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    tiles = "OSM",
    frame_width = 1600,
    frame_height = 900,
    size = "Humidity", 
    legend = "right"
)

# Display the map
plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [85]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] > 60) & (city_data_df["Max Temp"] < 70) & (city_data_df["Wind Speed"] < 8)]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,albany,42.6001,-73.9662,67.928,82,32,0.45,US,08/10/2023
24,24,taltal,-25.4000,-70.4833,65.048,63,63,1.07,CL,08/10/2023
29,29,plastun,44.7561,136.2939,62.474,99,100,2.85,RU,08/10/2023
41,41,tiksi,71.6872,128.8694,69.926,39,29,2.46,RU,08/10/2023
50,50,lompoc,34.6391,-120.4579,68.144,86,100,1.54,US,08/10/2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [86]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\meek\AppData\Local\Temp\ipykernel_12480\2603597672.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,87,
1,mersa matruh,EG,31.3525,27.2453,71,
2,northumberland,GB,55.2500,-2.0000,95,
3,bredasdorp,ZA,-34.5322,20.0403,87,
4,puerto natales,CL,-51.7236,-72.4875,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [87]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df["Lat"][index]
    lng = hotel_df["Lng"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
mersa matruh - nearest hotel: صقور
northumberland - nearest hotel: Newcastle House Hotel
bredasdorp - nearest hotel: Victoria Hotel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
inirida - nearest hotel: Residencias El Turista del Guainia
isafjordur - nearest hotel: Hótel Horn
hermanus - nearest hotel: Aloe guest house
egilsstadir - nearest hotel: Valaskjálf
colonia - nearest hotel: Wasserturm Hotel Cologne
west island - nearest hotel: Cocos Village Bungalows
iskateley - nearest hotel: Авантаж
buala - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas
vorgashor - nearest hotel: София
hawaiian paradise park - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
adamstown - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
olonkinbyen - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
burang 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,87,Keravel
1,mersa matruh,EG,31.3525,27.2453,71,صقور
2,northumberland,GB,55.2500,-2.0000,95,Newcastle House Hotel
3,bredasdorp,ZA,-34.5322,20.0403,87,Victoria Hotel
4,puerto natales,CL,-51.7236,-72.4875,94,Dorotea Patagonia Hostel
...,...,...,...,...,...,...
559,sirte,LY,31.2089,16.5887,61,فندق المدينة-سرت
560,pierre,US,44.3683,-100.3510,61,Quality Inn
561,gilgit,PK,35.9221,74.3087,42,PTDC Motel
562,tucurui,BR,-3.7661,-49.6725,90,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [89]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,87,Keravel
1,mersa matruh,EG,31.3525,27.2453,71,صقور
2,northumberland,GB,55.2500,-2.0000,95,Newcastle House Hotel
3,bredasdorp,ZA,-34.5322,20.0403,87,Victoria Hotel
4,puerto natales,CL,-51.7236,-72.4875,94,Dorotea Patagonia Hostel
...,...,...,...,...,...,...
559,sirte,LY,31.2089,16.5887,61,فندق المدينة-سرت
560,pierre,US,44.3683,-100.3510,61,Quality Inn
561,gilgit,PK,35.9221,74.3087,42,PTDC Motel
562,tucurui,BR,-3.7661,-49.6725,90,No hotel found


In [96]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    tiles = "OSM",
    frame_width = 1600,
    frame_height = 900,
    size = "Humidity", 
    legend = "right",
    hover_cols=['Country','Hotel Name']
)

# Display the map
plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)